# செமாண்டிக் கர்னல்

இந்த குறியீட்டு மாதிரியில், நீங்கள் [செமாண்டிக் கர்னல்](https://aka.ms/ai-agents-beginners/semantic-kernel) AI Framework-ஐப் பயன்படுத்தி ஒரு அடிப்படை முகவரியை உருவாக்குவீர்கள்.

இந்த மாதிரியின் நோக்கம், பின்னர் வேறு முகவரியின் முறைமைகளை செயல்படுத்தும் போது பயன்படுத்தப்படும் படிகளை உங்களுக்கு காட்டுவதாகும்.


## தேவையான பைதான் தொகுப்புகளை இறக்குமதி செய்யவும்


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## கிளையன்ட் உருவாக்குதல்

இந்த மாதிரியில், [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ஐ LLM-க்கு அணுகுவதற்காக பயன்படுத்துவோம்.

`ai_model_id` என்பது `gpt-4o-mini` என வரையறுக்கப்பட்டுள்ளது. GitHub Models சந்தையில் கிடைக்கும் மற்றொரு மாடலுக்கு மாறி, வெவ்வேறு முடிவுகளைப் பார்க்க முயற்சிக்கவும்.

GitHub Models க்கான `base_url` ஐ பயன்படுத்துவதற்காக `Azure Inference SDK` ஐ பயன்படுத்த வேண்டும். Semantic Kernel இல் உள்ள `OpenAIChatCompletion` இணைப்பியை பயன்படுத்துவோம். Semantic Kernel ஐ மற்ற மாடல் வழங்குநர்களுக்கு பயன்படுத்துவதற்கான [கிடைக்கக்கூடிய இணைப்பிகள்](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) உள்ளன.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## ஏஜென்டை உருவாக்குதல்

கீழே, `TravelAgent` எனப்படும் ஏஜென்டை உருவாக்குகிறோம்.

இந்த எடுத்துக்காட்டில், மிகவும் எளிய வழிமுறைகளை பயன்படுத்துகிறோம். இந்த வழிமுறைகளை மாற்றி, ஏஜென்ட் எப்படி வேறுபடுகிறதென்று பார்க்கலாம்.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## ஏஜென்டை இயக்குதல்

இப்போது `ChatHistoryAgentThread` வகை த்ரெட்டை வரையறுத்து ஏஜென்டை இயக்கலாம். தேவையான சிஸ்டம் செய்திகளை ஏஜென்டின் invoke_stream இன் `messages` கீவேர்ட் ஆர்க்யூமென்டில் வழங்கலாம்.

இவை வரையறுக்கப்பட்ட பிறகு, `user_inputs` உருவாக்குகிறோம், இது பயனர் ஏஜென்டுக்கு அனுப்பும் தகவலாக இருக்கும். இந்தச் சூழலில், இந்த செய்தியை `Plan me a sunny vacation` என்று அமைத்துள்ளோம்.

ஏஜென்டின் பதில் எப்படி மாறுகிறது என்பதைப் பார்க்க இந்த செய்தியை மாற்றுவதற்கு தயங்க வேண்டாம்.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கின்றோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறுகள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளுங்கள். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
